In [1]:
%matplotlib inline
    
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor 

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# final df for model building
df1 = pd.read_csv("ML_model_wk40_to_20.csv")
df1.dropna(how='any', inplace=True)
del df1['ILI_weeks']
del df1['Unnamed: 0']
#del df1['week']
df1.head(2)

Year  week  a_influenza  acute_bronchitis  body_temperature  \
0  2009    40           44                33                81   
1  2009    41           51                51                77   

   braun_thermoscan  break_a_fever  bronchitis  chest_cold  cold_and_flu  \
0                69             22          40          35            36   
1                46             24          43          35            43   

      ...      walking_pneumonia  what_to_do_if_you_have_the_flu  \
0     ...                     47                              56   
1     ...                     53                              58   

   Flu_Visit_Count  ILI_Visit_Count  Unspecified  CDC_Unweighted_ILI  \
0          0.01338          0.01763      0.03074             5.66087   
1          0.01620          0.02103      0.03554             6.81522   

   ILI_lagwk1  ILI_lagwk2  ILI_lagwk3  ILI_lagwk4  
0     6.81522     7.61889     7.38836     6.33927  
1     7.61889     7.38836     6.33927     4.94349  

[2 rows x 139 columns]

In [4]:
#  combining year and week column 
def combine_year_week(row):
    return int(row["Year"]) * 100 + int(row["week"])

In [5]:
df1["YearWeek"] = df1.apply(combine_year_week, axis=1)

In [6]:
# reset index to YearWeek
df1.index = df1["YearWeek"]

In [7]:
model_df = df1.drop(["Year", "week", "YearWeek"], axis=1)
model_df.head(2)

a_influenza  acute_bronchitis  body_temperature  braun_thermoscan  \
YearWeek                                                                      
200940             44                33                81                69   
200941             51                51                77                46   

          break_a_fever  bronchitis  chest_cold  cold_and_flu  cold_or_flu  \
YearWeek                                                                     
200940               22          40          35            36           37   
200941               24          43          35            43           49   

          cold_versus_flu     ...      walking_pneumonia  \
YearWeek                      ...                          
200940                 30     ...                     47   
200941                 41     ...                     53   

          what_to_do_if_you_have_the_flu  Flu_Visit_Count  ILI_Visit_Count  \
YearWeek                                                                     
200940                                56          0.01338          0.01763   
200941                                58          0.01620          0.02103   

          Unspecified  CDC_Unweighted_ILI  ILI_lagwk1  ILI_lagwk2  ILI_lagwk3  \
YearWeek                                                                        
200940        0.03074             5.66087     6.81522     7.61889     7.38836   
200941        0.03554             6.81522     7.61889     7.38836     6.33927   

          ILI_lagwk4  
YearWeek              
200940       6.33927  
200941       4.94349  

[2 rows x 137 columns]

# Independent variables
        # 3 independent variables from athena EHR 
            # [(flu visit counts)/ (total patient visit counts) 
            # (ILI visit counts)/ (total patient visit counts)
            # (unspecified viral or ILI visit counts)/ (total patient visit counts)]
        # CDC historical CDC_Unweighted_ILI values: collected from 2009 to 2016 (week 40 to 20)
        # 129 google search terms related to flu
        
      ======> 3 + 1 + 129
      
# Dependent variables 
        # 4 ILI weeks offset by 1 week

# split df to train dataset, dataset before year2015 and week 40 used at training dataset

In [8]:
train = model_df[model_df.index < 201540]
# train.head()


# defining  targets/labels to  y axis
                y_train = ILI lag week 1

                yy_train = ILI lag week 2

                yyy_train = ILI lag week 3

                yyyy_train = ILI lag week 4

In [9]:
y_train = train["ILI_lagwk1"]
# y_train

In [10]:
yy_train = train["ILI_lagwk2"]

In [11]:
yyy_train = train["ILI_lagwk3"]

In [12]:
yyyy_train =train["ILI_lagwk4"]

In [13]:
X_train = train.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)
# X_train

# split df to train dataset,dataset after year2015 and week 40 used at training dataset

In [14]:
test = model_df[model_df.index >= 201540]
# test.head()

In [15]:
y_test = test["ILI_lagwk1"]

In [16]:
yy_test = test["ILI_lagwk2"]

In [17]:
yyy_test = test["ILI_lagwk3"]

In [18]:
yyyy_test = test["ILI_lagwk4"]

In [19]:
X_test = test.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)

# Reference https://shankarmsy.github.io/stories/gbrt-sklearn.html#

##https://www.youtube.com/watch?v=IXZKgIsZRm0

# building a model for y_test = test["ILI_lagwk1"]

In [20]:
gbrt1 = GradientBoostingRegressor(n_estimators = 100, max_depth = 5, min_samples_leaf = 9, 
                                  random_state = 0)

In [21]:
gbrt1.fit(X_train, y_train) 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=9,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [22]:
y_pred = gbrt1.predict(X_test)

In [23]:
r2_score(y_test, y_pred)

0.8454758558792845

In [24]:
mean_absolute_error(y_test, y_pred)

0.17756484904641834

In [25]:
print("R-squared for ILI_lagwk1 Train: %.2f" %gbrt1.score(X_train, y_train)) 
print("R-squared for ILI_lagwk1 Test : %.2f" %gbrt1.score(X_test, y_test)) 

R-squared for ILI_lagwk1 Train: 1.00
R-squared for ILI_lagwk1 Test : 0.85


In [26]:
predictions1 = gbrt1.predict(X_test) 

In [27]:
# predict the values of y
y_test_unraveled = y_test.values.ravel()
y_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [28]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions1, "Actual": y_test_unraveled}).reset_index(drop=True)
pred_df.index = y_test.index
pred_df.head()

Prediction_ILI_lagwk1   Actual
YearWeek                                
201540                 1.404117  1.33029
201541                 1.371077  1.41368
201542                 1.478061  1.36882
201543                 1.419678  1.48309
201544                 1.482375  1.54494

# # Create the GridSearchCV model

In [29]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
#             'max_features':[0.1, 0.3, 1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# selecting important features  ["ILI_lagwk1"]

In [30]:
# # GBRF predictors variable importance
# # store most important variables under importances
# importances = gbrt.feature_importances_

In [31]:
# gbrt.feature_importances_.sum()

In [32]:
# # store most important variables/features under importances
# importances = gbrt.feature_importances_
# # importances

In [33]:
# sorted_features = sorted(zip(importances, X_train.columns), reverse=True)
# # sorted_features

In [34]:
# # sort important varibles and sotre them under indices
# important_columns = []
# for fimportance, name in sorted(zip(importances, X_train.columns), reverse=True):
#     if fimportance > 0.00009:
#         important_columns.append(name)

In [35]:
# important_feature_columns = important_columns
# # important_feature_columns

In [36]:
# important_features_df = model_df[important_feature_columns]
# important_features_df.head()

# defining important feature X train and X test dataset

In [37]:
# X_trainim = important_features_df[important_features_df.index < 201540]
# # trainim
# # .head(2)

In [38]:
# X_testim = important_features_df[important_features_df.index >= 201540]
# # testim
# # .head()

In [39]:
# gbrtim_ili_lag1 = GradientBoostingRegressor(n_estimators = 500, max_depth = 5) # number of sequential trees to be modeled

In [40]:
# gbrtim_ili_lag1.fit(X_trainim, y_train) 

In [41]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [42]:
# r2_score(y_test, y_pred)

In [43]:
# mean_absolute_error(y_test, y_pred)

In [44]:
# # GBR with important features for ili_lag1 week 1
# print("R-squared for Train gbrtim_ili_lag1: %.2f" %gbrtim_ili_lag1.score(X_trainim, y_train)) 
# print("R-squared for Test gbrtim_ili_lag1: %.2f" %gbrtim_ili_lag1.score(X_testim, y_test)) 

In [45]:
# # predict the values of y with important feature extraction for ili week 1
# predictions = gbrtim_ili_lag1.predict(X_testim)
# y_test_unraveled = y_test.values.ravel()
# y_test.index

In [46]:
# Make predictions using the X_test and y_test data
# Print at least 10 predictions vs their actual labels
# predictions = gbrtim_ili_lag1.predict(X_testim)
# print(f"First 10 Predictions: {predictions[:10]}")
# print(f"First 10 Actual labels: {y_test_unraveled[:10]}")

In [47]:
# # Print predictions vs their actual labels
# pred_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test_unraveled}).reset_index(drop=True)
# pred_df.index = y_test.index
# pred_df.head()

In [48]:
# # Create the GridSearchCV model

# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
# #             'max_features':[0.1,0.3,1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# building a model for y_test = test["ILI_lagwk2"]

In [49]:
gbrt2 = GradientBoostingRegressor(n_estimators = 100, max_depth = 5, min_samples_leaf = 9, 
                                  random_state = 0)

In [50]:
gbrt2.fit(X_train, yy_train) 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=9,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [51]:
yy_pred = gbrt2.predict(X_test)

In [52]:
r2_score(yy_test, yy_pred)

0.32704292682891745

In [53]:
mean_absolute_error(yy_test, yy_pred)

0.34707173472660413

In [54]:
print("R-squared for ILI_lagwk2 Train: %.2f" %gbrt2.score(X_train, yy_train)) 
print("R-squared for ILI_lagwk2 Test: %.2f" %gbrt2.score(X_test, yy_test)) 

R-squared for ILI_lagwk2 Train: 1.00
R-squared for ILI_lagwk2 Test: 0.33


In [55]:
predictions2 = gbrt2.predict(X_test) 

In [56]:
# predict the values of y
yy_test_unraveled = yy_test.values.ravel()
yy_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [57]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk2": predictions2, "Actual": yy_test_unraveled}).reset_index(drop=True)
pred_df.index = yy_test.index
pred_df.head()

Prediction_ILI_lagwk2   Actual
YearWeek                                
201540                 1.384344  1.41368
201541                 1.437949  1.36882
201542                 1.450673  1.48309
201543                 1.677382  1.54494
201544                 1.559731  1.62532

# selecting important features  ["ILI_lagwk2"]

# defining important feature X train and X test dataset

# building a model for y_test = test["ILI_lagwk3"]

In [58]:
gbrt3 = GradientBoostingRegressor(n_estimators = 100, max_depth = 5, min_samples_leaf = 9, 
                                  random_state = 0)

In [59]:
gbrt3.fit(X_train, yyy_train) 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=9,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [60]:
yyy_pred = gbrt3.predict(X_test)

In [61]:
r2_score(yyy_test, yyy_pred)

0.054639369128333226

In [62]:
mean_absolute_error(yyy_test, yyy_pred)

0.4474042198926967

In [63]:
print("R-squared for Train: %.2f" %gbrt3.score(X_train, yyy_train)) 
print("R-squared for Test: %.2f" %gbrt3.score(X_test, yyy_test)) 

R-squared for Train: 0.99
R-squared for Test: 0.05


In [64]:
predictions3 = gbrt3.predict(X_test) 

In [65]:
# predict the values of y
predictions = gbrt3.predict(X_test)
yyy_test_unraveled = yyy_test.values.ravel()
yyy_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [66]:
pred_df = pd.DataFrame({"Prediction_ILI_lagwk3": predictions3, "Actual": yyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyy_test.index
pred_df.head()

Prediction_ILI_lagwk3   Actual
YearWeek                                
201540                 1.422373  1.36882
201541                 1.770672  1.48309
201542                 1.836371  1.54494
201543                 1.874872  1.62532
201544                 1.567445  1.91565

# building a model for y_test = test["ILI_lagwk4"]

In [67]:
gbrt4 = GradientBoostingRegressor(n_estimators = 100, max_depth = 5, min_samples_leaf = 9, 
                                  random_state = 0)

In [68]:
gbrt4.fit(X_train, yyyy_train) 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=9,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [69]:
yyyy_pred = gbrt4.predict(X_test)

In [70]:
r2_score(yyyy_test, yyyy_pred)

0.17261243837183193

In [71]:
mean_absolute_error(yyyy_test, yyyy_pred)

0.43632068751861675

In [72]:
print("R-squared for Train: %.2f" %gbrt4.score(X_train, yyyy_train)) 
print("R-squared for Test: %.2f" %gbrt4.score(X_test, yyyy_test)) 

R-squared for Train: 1.00
R-squared for Test: 0.17


In [73]:
# predict the values of y
predictions4 = gbrt4.predict(X_test)
yyyy_test_unraveled = yyyy_test.values.ravel()
yyyy_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [74]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions4, "Actual": yyyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyyy_test.index
pred_df.head()

Prediction_ILI_lagwk1   Actual
YearWeek                                
201540                 1.739046  1.48309
201541                 1.930571  1.54494
201542                 1.957137  1.62532
201543                 1.910867  1.91565
201544                 2.296120  1.74368